### 決定木の複雑さの制御

一般に、上で述べたような方法で、葉が純粋になるまで分割を続けると、モデルは複雑になりすぎ、訓練データに対して大幅に過剰適合してしまう。葉が純粋になっているということは、訓練セットに対しては100%の精度になっているということである。訓練セットのデータポイントは、俗数はの中で常に正しい多数派クラスになっているからだ。**図2-26**を見れば過剰適合が起きていることがわかる。クラス０の領域の中にクラス１に属する領域がある。これは多くの人が期待する決定境界とは異なるだろう。この決定境界は、そのクラスに属する他の点からかけ離れた場所に１つだけある外れ値を重視しすぎている。

過剰適合を防ぐには２つの戦略がある。構築過程で木の生成を速めに止める**事前枝刈り**（pre-pruning）と、一度木を構築してから、情報の少ないノードを削除する**事後枝刈り**（post-pruning）（ただの**枝刈り**（pruning）とも呼ばれる）である。事前枝刈りの方法としては、木の深さを制限する方法や、葉の最大値を制限する方法、分割する際にその中に含まれている点の最小値を決めておく方法がある。

scikit-learnでは、決定木はDecisionTreeRegressorクラスとDecisionTreeClassifierクラスに実装されている。scikit-learnには事前枝刈りしか実装されていない。事前枝刈りの効果を、cancerデータセットを用いてより詳しく見てみよう。いつものように、データセットを読み込んで訓練セットとデータセットに分割する。次にデフォルトの設定で完全な木を構築する（葉が純粋になるまで木を育てる）。ここでは、内部でタイブレークに使われるrandom_stateを固定している。

In [7]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import mglearn
from IPython.display import display

from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

cancer = load_breast_cancer()
X_train, X_test, y_train, y_test = train_test_split(
    cancer.data, cancer.target, stratify=cancer.target, random_state=42)
tree = DecisionTreeClassifier(random_state=0)
tree.fit(X_train, y_train)
print("Accuracy on training set: {:.3f}".format(tree.score(X_train, y_train)))
print("Accuracy on test set: {:.3f}".format(tree.score(X_test, y_test)))

Accuracy on training set: 1.000
Accuracy on test set: 0.937


予想通り、訓練セットの精度は100%である。これは葉が純粋で、訓練データのすべてのラベルを十分なほど木が育っているからだ。テストセットに対する精度は、以前見た線形モデルより少し悪く95%程度になっている。

決定木の深さに制約を与えないと、決定木はいくらでも深く、複雑になる。したがって、枝刈りされていない木は過剰適合になりやすく、新しいデータに対する汎化性能が低い傾向になる。ここで、事前枝刈りを適用して、木が完全に訓練データに適合する前に木の成長を止めてみよう。１つの方法は、木がある深さに達したらそこで止めるという方法だ。ここではmax_depth=4としている。こうすると質問の列は４つまで、ということになる。（**図2-24**、**図2-26**）。木の深さを制限することで過剰適合が抑制される。これによって、訓練制度は下がるが、テストセットに対する精度は向上する。

In [10]:
tree = DecisionTreeClassifier(max_depth=4, random_state=0)
tree.fit(X_train, y_train)

print("Accuracy on training set: {:.3f}".format(tree.score(X_train, y_train)))
print("Accuracy on test set: {:.3f}".format(tree.score(X_test, y_test)))

Accuracy on training set: 0.988
Accuracy on test set: 0.951
